In [1]:
import pandas as pd

df = pd.read_csv("top50_posts_per_user_reduced.csv")       # autor, trait, post, similarity
labels = pd.read_csv("authors_train.csv")  # username + 5 rasgos

df = df.merge(labels, on="username")

df.head()




,username,trait,post,similarity,agreeableness,openness,conscientiousness,extraversion,neuroticism
0,-Areopagan-,Agreeableness,I am smarter than you and will work you into d...,0.476548,0.0,99.0,96.0,60.0,1.0
1,-Areopagan-,Agreeableness,"I have two friends. I alienate everyone, event...",0.472431,0.0,99.0,96.0,60.0,1.0
2,-Areopagan-,Agreeableness,Yeah I wouldnt want to deal with someone like ...,0.390348,0.0,99.0,96.0,60.0,1.0
3,-Areopagan-,Agreeableness,your first and second question is the same que...,0.328232,0.0,99.0,96.0,60.0,1.0
4,-Areopagan-,Openness,I am smarter than you and will work you into d...,0.357194,0.0,99.0,96.0,60.0,1.0


In [2]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

In [2]:
#esta funcion lo que busca es indicar cual es la columna objetivo en cada fila, en base al trait de la columna trait lo que coge como etiqueta es la columna que se llame igual al valor que hay en trait
def get_target(row):
    return str(row[row["trait"].lower()])  # valor numérico correcto

given the post: {post} estimate the personality trait {trait} of the author. return only a number between 0 and 100.


df["input_text"] = df.apply(lambda row: f"predict this trait: {row['trait']}: {row['post']}", axis=1)
df["target_text"] = df.apply(get_target, axis=1)

In [3]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["input_text", "target_text"]])
dataset = dataset.train_test_split(test_size=0.1)


/home/usc/cursos/curso1589/.conda/envs/t5_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(example):
    enc = tokenizer(
        example["input_text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["target_text"],
            padding="max_length",
            truncation=True,
            max_length=10
        )
    enc["labels"] = labels["input_ids"]
    return enc

tokenized = dataset.map(preprocess, batched=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map:   0%|                                                                            | 0/139886 [00:00<?, ? examples/s]/home/usc/cursos/curso1589/.conda/envs/t5_env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.


In [9]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [10]:
training_args = TrainingArguments(
    output_dir="./t5_personality",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    report_to="none",
    learning_rate=3e-4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=2,
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
)


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()


Step,Training Loss
100,1.267700
200,0.519900
300,0.502000
400,0.489100
500,0.486100
600,0.481500
700,0.482000
800,0.477400
900,0.475100
1000,0.475500


In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("./t5_personality/checkpoint-24000")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [9]:
import torch
torch.cuda.is_available()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [10]:
#Función para predecir un post con T5
def predict_trait_post(model, tokenizer, trait, post):
    input_text = f"predict {trait}: {post}"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(model.device)

    outputs = model.generate(**inputs, max_length=5)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    try:
        return float(pred)
    except:
        return np.nan



In [12]:
from tqdm import tqdm
import numpy as np

posts = df["post"].tolist()
traits = df["trait"].tolist()

batch_size = 16
preds = []

for i in tqdm(range(0, len(posts), batch_size), desc="Generando predicciones"):
    batch_posts = posts[i:i+batch_size]
    batch_traits = traits[i:i+batch_size]
    
    input_texts = [f"predict {t}: {p}" for t, p in zip(batch_traits, batch_posts)]

    inputs = tokenizer(
        input_texts,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=5)

    batch_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for pred in batch_preds:
        try:
            preds.append(float(pred.strip()))
        except:
            preds.append(np.nan)

df["predicted"] = preds


Generando predicciones: 100%|███████████████████████████████████████████████████████| 9715/9715 [12:02<00:00, 13.45it/s]


In [13]:
#Comparar cada predicción con la columna correcta
df["target_value"] = df.apply(
    lambda row: row[row["trait"].lower()],
    axis=1
)


In [20]:
df[df["predicted"].isna()].head()


,username,trait,post,similarity,agreeableness,openness,conscientiousness,extraversion,neuroticism,input_text,target_text,predicted,target_value


In [17]:
df[df["target_value"].isna()].head()


,username,trait,post,similarity,agreeableness,openness,conscientiousness,extraversion,neuroticism,input_text,target_text,predicted,target_value
2069,Aiichai,Openness,"My ideas are processed quickly in quantity, an...",0.441203,26.0,NaN,23.0,71.0,70.0,predict this trait: Openness: My ideas are pro...,nan,85.0,NaN
2070,Aiichai,Openness,"My ideas are processed quickly in quantity, an...",0.415770,26.0,NaN,23.0,71.0,70.0,predict this trait: Openness: My ideas are pro...,nan,85.0,NaN
2071,Aiichai,Openness,Hmm. Openness - 100%Conscientious - 44%Extrav...,0.358182,26.0,NaN,23.0,71.0,70.0,predict this trait: Openness: Hmm. Openness - ...,nan,85.0,NaN
2072,Aiichai,Openness,I'm so mad because I relate to this to a molec...,0.352888,26.0,NaN,23.0,71.0,70.0,predict this trait: Openness: I'm so mad becau...,nan,85.0,NaN
2073,Aiichai,Openness,I wrote about myself and things i held back sa...,0.321436,26.0,NaN,23.0,71.0,70.0,predict this trait: Openness: I wrote about my...,nan,85.0,NaN


In [18]:
df_clean = df.dropna(subset=["predicted", "target_value"])

In [19]:
#Calcular RMSE por trait

from sklearn.metrics import mean_squared_error
import numpy as np

traits = ["openness", "conscientiousness", "extraversion",
          "agreeableness", "neuroticism"]

rmse_by_trait = {}

for t in traits:
    subset = df_clean[df_clean["trait"].str.lower() == t]

    if len(subset) == 0:
        rmse_by_trait[t] = None
        continue

    mse = mean_squared_error(subset["target_value"], subset["predicted"])
    rmse = np.sqrt(mse)

    rmse_by_trait[t] = rmse

rmse_by_trait




{'openness': np.float64(35.0935055586072),
 'conscientiousness': np.float64(49.052841481160065),
 'extraversion': np.float64(46.141689043071324),
 'agreeableness': np.float64(51.720501972611586),
 'neuroticism': np.float64(55.89428075583088)}